<a href="https://colab.research.google.com/github/SorokinMaksimArtemovich/MTS-ML-CUP/blob/main/data%20preprocessing/0.0_bace_datapreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this nootebook I use this features for aggregation, making matrix and making ALS embanding:

- region_name
- city_name
- cpe_manufacturer_name
- cpe_model_name
- cpe_type_cd
- cpe_model_os_type
- date
- part_of_day

In [ ]:
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import gc
import time
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
import pickle
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
!pip install feather-format >> none
!pip install faiss-cpu --no-cache
!pip install implicit
import implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 196.7 MB/s eta 0:00:00


In [ ]:
data = pd.read_feather('/data/dataset_full.feather')
data = pa.Table.from_pandas(data)

# Baceline
Baceline by organizers

In [ ]:
%%time
data_agg = data.select(['user_id', 'url_host', 'request_cnt']).\
    group_by(['user_id', 'url_host']).aggregate([('request_cnt', 'sum')])

CPU times: user 24.9 s, sys: 5.89 s, total: 30.8 s
Wall time: 30.7 s


In [ ]:
url_set = set(data_agg.select(['url_host']).to_pandas()['url_host'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(data_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

199683 urls
415317 users


In [ ]:
%%time
values = np.array(data_agg.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(data_agg.select(['url_host']).to_pandas()['url_host'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 100, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1, nlist=400, nprobe=20)

CPU times: user 1.99 s, sys: 852 ms, total: 2.84 s
Wall time: 2.77 s


In [ ]:
%%time
als.fit(mat)

  0%|          | 0/100 [00:00<?, ?it/s]

CPU times: user 1h 12min 21s, sys: 38.1 s, total: 1h 12min 59s
Wall time: 18min 54s


In [ ]:
d_factors = als.item_factors
u_factors = als.user_factors

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
bace_emb = pd.DataFrame(d_factors)
bace_emb['user_id'] = bace_emb.index.map(inv_usr_map)

CPU times: user 463 ms, sys: 95.9 ms, total: 559 ms
Wall time: 558 ms


In [ ]:
bace_emb.to_csv('data/bace_preprocessing/df_bace.csv', index = False)

In [ ]:
del(data_agg)
del(values)
del(rows)
del(cols)
del(mat)
del(als)
del(d_factors)
del(inv_usr_map)
del(bace_emb)
gc.collect()

21

### Saving url-embending for url encoding

In [ ]:
def save(obj, path, verbose=True):
    if verbose:
        print("Saving object to {}".format(path))

    with open(path, "wb") as obj_file:
        pickle.dump( obj, obj_file, protocol=pickle.HIGHEST_PROTOCOL )

    if verbose:
        print("Object saved to {}".format(path))
    pass

url_factor_features_dict = {}
url_factor_features_dict["feature_names"] = []
for i in range(len(u_factors[0])):
    url_factor_features_dict["feature_names"].append( "url_factor_{}".format(i) )
uniq_urls = list(sorted(list(set(data.select(['url_host']).to_pandas()['url_host'].values))))
url_id_dict = {}
for i in tqdm(range( len(uniq_urls) ), desc="Building url --> id mapping"):
    url_id_dict[ uniq_urls[i] ] = i
inverted_url_id_dict = {v: k for k, v in url_id_dict.items()}
for i in tqdm(range( len(u_factors) ), desc="Building url factor features dict"):
    url_id = inverted_url_id_dict[i]
    url_factor_features = u_factors[i]
    url_factor_features_dict[ url_id ] = url_factor_features

save(url_factor_features_dict, "data/utils/url_only_factor_features_dict.pkl")

Building url factor features dict: 100%|██████████| 199683/199683 [00:00<00:00, 229057.28it/s]


Saving object to url_only_factor_features_dict.pkl
Object saved to url_only_factor_features_dict.pkl


# ALS embandings

## Count of day that urers visit url_hosts
I count the number of non-repetitive days for each user visit each url_host and use it as weight in sparse matrix for ALS embanding

In [ ]:
%%time
data_agg = pa.Table.from_pandas(data.select(['user_id', 'url_host', 'date']).\
    to_pandas().drop_duplicates()).\
    group_by(['user_id', 'url_host']).aggregate([('date', 'count')])

CPU times: user 2min 6s, sys: 29.3 s, total: 2min 35s
Wall time: 2min 28s


In [ ]:
%%time
values = np.array(data_agg.select(['date_count']).to_pandas()['date_count'])
rows = np.array(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(data_agg.select(['url_host']).to_pandas()['url_host'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

CPU times: user 2.01 s, sys: 952 ms, total: 2.96 s
Wall time: 2.77 s


In [ ]:
%%time
als.fit(mat)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 22min 6s, sys: 11.1 s, total: 22min 17s
Wall time: 5min 48s


In [ ]:
d_factors = als.item_factors

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
date_emb = pd.DataFrame(d_factors)
date_emb['user_id'] = date_emb.index.map(inv_usr_map)

CPU times: user 458 ms, sys: 25 ms, total: 483 ms
Wall time: 483 ms


In [ ]:
date_emb.to_csv('data/bace_preprocessing/date_emb.csv', index = False)

In [ ]:
del(data_agg)
del(values)
del(rows)
del(cols)
del(mat)
del(als)
del(d_factors)
del(inv_usr_map)
del(date_emb)
del(url_set)
del(url_dict)
del(usr_set)
del(usr_dict)
gc.collect()

21

## Region
I count the number of requests for each user from each region and use it as weight in sparse matrix for ALS embanding

In [ ]:
%%time
region_agg = data.select(['user_id', 'region_name', 'request_cnt']).\
    group_by(['user_id', 'region_name']).aggregate([('request_cnt', 'count')])

CPU times: user 10.3 s, sys: 781 ms, total: 11.1 s
Wall time: 11.1 s


In [ ]:
usr_set = set(region_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}
region_set = set(region_agg.select(['region_name']).to_pandas()['region_name'])
print(f'{len(region_set)} regions')
region_dict = {url: idurl for url, idurl in zip(region_set, range(len(region_set)))}

415317 users
81 regions


In [ ]:
%%time
values = np.array(region_agg.select(['request_cnt_count']).to_pandas()['request_cnt_count'])
rows = np.array(region_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(region_agg.select(['region_name']).to_pandas()['region_name'].map(region_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

CPU times: user 410 ms, sys: 6.04 ms, total: 416 ms
Wall time: 414 ms


In [ ]:
%%time
als.fit(mat)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 3min 28s, sys: 51.5 s, total: 4min 19s
Wall time: 1min 8s


In [ ]:
d_factors = als.item_factors

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
region_emb = pd.DataFrame(d_factors)
region_emb['user_id'] = region_emb.index.map(inv_usr_map)

CPU times: user 454 ms, sys: 34 ms, total: 488 ms
Wall time: 487 ms


In [ ]:
region_emb.to_csv('data/bace_preprocessing/region_emb.csv', index = False)

In [ ]:
del(region_agg)
del(values)
del(rows)
del(cols)
del(mat)
del(als)
del(d_factors)
del(inv_usr_map)
del(region_emb)
del(region_set)
del(region_dict)
gc.collect()

21

## City
I count the number of requests for each user from each city and use it as weight in sparse matrix for ALS embanding

In [ ]:
%%time
city_agg = data.select(['user_id', 'region_name', 'city_name', 'request_cnt']).\
    group_by(['user_id', 'region_name', 'city_name']).aggregate([('request_cnt', 'count')])

CPU times: user 11.4 s, sys: 800 ms, total: 12.2 s
Wall time: 12.2 s


In [ ]:
city_agg = city_agg.to_pandas()
city_agg['city_name'] = city_agg['region_name'].astype('string') + ' ' + city_agg['city_name'].astype('string')
city_agg = city_agg.drop('region_name', axis=1)
city_agg

,request_cnt_count,user_id,city_name
0,1543,45098,Краснодарский край Краснодар
1,4,45098,Ставропольский край Ставрополь
2,1,45098,Республика Адыгея Адыгейск
3,1,45098,Ставропольский край Изобильный
4,1,45098,Ставропольский край Новоалександровск
...,...,...,...
1288654,236,260127,Курская область Курск
1288655,623,260127,Белгородская область Белгород
1288656,1,260127,Белгородская область Строитель
1288657,1110,300964,Калужская область Обнинск


In [ ]:
city_agg = pa.Table.from_pandas(city_agg)

In [ ]:
city_set = set(city_agg.select(['city_name']).to_pandas()['city_name'])
print(f'{len(city_set)} cities')
city_dict = {url: idurl for url, idurl in zip(city_set, range(len(city_set)))}

1000 cities


In [ ]:
%%time
values = np.array(city_agg.select(['request_cnt_count']).to_pandas()['request_cnt_count'])
rows = np.array(city_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(city_agg.select(['city_name']).to_pandas()['city_name'].map(city_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

CPU times: user 1.06 s, sys: 121 ms, total: 1.18 s
Wall time: 1.18 s


In [ ]:
%%time
als.fit(mat)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 3min 44s, sys: 47.4 s, total: 4min 31s
Wall time: 1min 12s


In [ ]:
d_factors = als.item_factors

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
city_emb = pd.DataFrame(d_factors)
city_emb['user_id'] = city_emb.index.map(inv_usr_map)

CPU times: user 1.27 s, sys: 91.4 ms, total: 1.37 s
Wall time: 1.37 s


In [ ]:
city_emb.to_csv('data/bace_preprocessing/city_emb.csv', index = False)

In [ ]:
del(city_agg)
del(values)
del(rows)
del(cols)
del(mat)
del(als)
del(d_factors)
del(inv_usr_map)
del(city_emb)
del(city_set)
del(city_dict)
gc.collect()

21

## Model
I count the number of requests for each user from each model and use it as weight in sparse matrix for ALS embanding

In [ ]:
%%time
model_agg = data.select(['user_id', 'cpe_model_name', 'request_cnt']).\
    group_by(['user_id', 'cpe_model_name']).aggregate([('request_cnt', 'count')])

CPU times: user 10.2 s, sys: 2.04 s, total: 12.3 s
Wall time: 12.3 s


In [ ]:
model_set = set(model_agg.select(['cpe_model_name']).to_pandas()['cpe_model_name'])
print(f'{len(model_set)} cities')
model_dict = {url: idurl for url, idurl in zip(model_set, range(len(model_set)))}

599 cities


In [ ]:
%%time
values = np.array(model_agg.select(['request_cnt_count']).to_pandas()['request_cnt_count'])
rows = np.array(model_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(model_agg.select(['cpe_model_name']).to_pandas()['cpe_model_name'].map(model_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
als = implicit.approximate_als.FaissAlternatingLeastSquares(factors = 50, iterations = 30, use_gpu = False, \
       calculate_training_loss = False, regularization = 0.1)

CPU times: user 396 ms, sys: 155 ms, total: 551 ms
Wall time: 401 ms


In [ ]:
%%time
als.fit(mat)

  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: user 3min 7s, sys: 49.8 s, total: 3min 56s
Wall time: 1min 2s


In [ ]:
u_factors = als.user_factors 
d_factors = als.item_factors

In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
model_emb = pd.DataFrame(d_factors)
model_emb['user_id'] = model_emb.index.map(inv_usr_map)

CPU times: user 451 ms, sys: 30.9 ms, total: 482 ms
Wall time: 483 ms


In [ ]:
model_emb.to_csv('data/bace_preprocessing/model_emb.csv', index = False)

In [ ]:
del(model_agg)
del(values)
del(rows)
del(cols)
del(mat)
del(als)
del(d_factors)
del(inv_usr_map)
del(model_emb)
del(model_set)
del(model_dict)
del(usr_set)
del(usr_dict)
gc.collect()

21

# Matrix
I encode this features with count and sum of request_cnt

## PartOfDay

In [ ]:
%%time
data_agg = data.select(['user_id', 'part_of_day', 'request_cnt']).\
    group_by(['user_id', 'part_of_day']).aggregate([('request_cnt', 'sum'), ('request_cnt', 'count')])

CPU times: user 12.9 s, sys: 1.27 s, total: 14.2 s
Wall time: 14.2 s


In [ ]:
usr_set = set(data_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}
region_set = set(data_agg.select(['part_of_day']).to_pandas()['part_of_day'])
print(f'{len(region_set)} part_of_days')
region_dict = {url: idurl for url, idurl in zip(region_set, range(len(region_set)))}
values = np.array(data_agg.select(['request_cnt_count']).to_pandas()['request_cnt_count'])
rows = np.array(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(data_agg.select(['part_of_day']).to_pandas()['part_of_day'].map(region_dict))
count_mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1)).toarray()

415317 users
4 part_of_days


In [ ]:
usr_set = set(data_agg.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}
region_set = set(data_agg.select(['part_of_day']).to_pandas()['part_of_day'])
print(f'{len(region_set)} part_of_days')
region_dict = {url: idurl for url, idurl in zip(region_set, range(len(region_set)))}
values = np.array(data_agg.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(data_agg.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(data_agg.select(['part_of_day']).to_pandas()['part_of_day'].map(region_dict))
sum_mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1)).toarray()

415317 users
4 part_of_days


In [ ]:
%%time
inv_usr_map = {v: k for k, v in usr_dict.items()}
count_mat = pd.DataFrame(count_mat)
count_mat['user_id'] = count_mat.index.map(inv_usr_map)
sum_mat = pd.DataFrame(sum_mat)
sum_mat['user_id'] = sum_mat.index.map(inv_usr_map)
count_mat = count_mat.merge(sum_mat, on='user_id', how='inner', suffixes=('count', 'sum'))
count_mat.to_csv('data/bace_preprocessing/part_of_day.csv', index=False)

CPU times: user 1.98 s, sys: 41.9 ms, total: 2.02 s
Wall time: 2.03 s


In [ ]:
del(data_agg)
del(values)
del(rows)
del(cols)
del(sum_mat)
del(count_mat)
del(inv_usr_map)
del(region_set)
del(region_dict)
del(usr_set)
del(usr_dict)
gc.collect()

63

# AGG
I aggregate counts and sums of request_cnt by features and user and count mean, median, min, max and standard deviation od it by each user

### date

In [ ]:
%%time
date_agg = data.select(['user_id', 'date', 'request_cnt']).\
    group_by(['user_id', 'date']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('date', 'count'), ('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 23.6 s, sys: 8.03 s, total: 31.7 s
Wall time: 31.5 s


In [ ]:
date_agg.to_pandas().to_csv('data/bace_preprocessing/date_agg.csv', index=False)

In [ ]:
del(date_agg)
gc.collect()

63

### url_host

In [ ]:
%%time
url_agg = data.select(['user_id', 'url_host', 'request_cnt']).\
    group_by(['user_id', 'url_host']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 32.6 s, sys: 2.49 s, total: 35 s
Wall time: 34.7 s


In [ ]:
url_agg.to_pandas().to_csv('data/bace_preprocessing/url_agg.csv', index=False)

In [ ]:
del(url_agg)
gc.collect()

63

### region

In [ ]:
%%time
region_agg = data.select(['user_id', 'region_name', 'request_cnt']).\
    group_by(['user_id', 'region_name']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 13 s, sys: 815 ms, total: 13.8 s
Wall time: 13.7 s


In [ ]:
region_agg.to_pandas().to_csv('data/bace_preprocessing/region_agg.csv', index=False)

In [ ]:
del(region_agg)
gc.collect()

63

### city

In [ ]:
%%time
city_agg = data.select(['user_id', 'region_name', 'city_name', 'request_cnt']).\
    group_by(['user_id', 'region_name', 'city_name']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 14.2 s, sys: 933 ms, total: 15.1 s
Wall time: 15 s


In [ ]:
city_agg.to_pandas().to_csv('data/bace_preprocessing/city_agg.csv', index=False)

In [ ]:
del(city_agg)
gc.collect()

63

### model

In [ ]:
%%time
model_agg = data.select(['user_id', 'cpe_manufacturer_name', 'cpe_model_name', 'request_cnt']).\
    group_by(['user_id', 'cpe_manufacturer_name', 'cpe_model_name']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 13.8 s, sys: 862 ms, total: 14.6 s
Wall time: 14.5 s


In [ ]:
model_agg.to_pandas().to_csv('data/bace_preprocessing/model_agg.csv', index=False)

In [ ]:
del(model_agg)
gc.collect()

63

### manufacturer_name

In [ ]:
%%time
manuf_agg = data.select(['user_id', 'cpe_manufacturer_name', 'request_cnt']).\
    group_by(['user_id', 'cpe_manufacturer_name']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 12.7 s, sys: 868 ms, total: 13.6 s
Wall time: 13.4 s


In [ ]:
manuf_agg.to_pandas().to_csv('data/bace_preprocessing/manuf_agg.csv', index=False)

In [ ]:
del(manuf_agg)
gc.collect()

63

### cpe_type

In [ ]:
%%time
cpe_agg = data.select(['user_id', 'cpe_type_cd', 'request_cnt']).\
    group_by(['user_id', 'cpe_type_cd']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 13.6 s, sys: 859 ms, total: 14.4 s
Wall time: 14.3 s


In [ ]:
cpe_agg.to_pandas().to_csv('data/bace_preprocessing/cpe_agg.csv', index=False)

In [ ]:
del(cpe_agg)
gc.collect()

63

### os_type

In [ ]:
%%time
os_agg = data.select(['user_id', 'cpe_model_os_type', 'request_cnt']).\
    group_by(['user_id', 'cpe_model_os_type']).aggregate([('request_cnt', 'count'), ('request_cnt', 'sum')]).\
    group_by(['user_id']).aggregate([('request_cnt_count', 'mean'), ('request_cnt_count', 'approximate_median'), ('request_cnt_count', 'min'), ('request_cnt_count', 'max'), ('request_cnt_count', 'stddev'), ('request_cnt_sum', 'mean'), ('request_cnt_sum', 'approximate_median'), ('request_cnt_sum', 'min'), ('request_cnt_sum', 'max'), ('request_cnt_sum', 'stddev')])

CPU times: user 12.5 s, sys: 896 ms, total: 13.4 s
Wall time: 13.3 s


In [ ]:
os_agg.to_pandas().to_csv('data/bace_preprocessing/os_agg.csv', index=False)

In [ ]:
del(os_agg)
gc.collect()

63